### Manipulation of Comet database search results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Comet search results (.csv) of PTM-optimized database searches
    Includes precursor intensities and protein descriptions

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported Comet search results CSVs into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTMopt-hyroxylation/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTMopt-hyroxylation


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls

20170410_JA4_01_quant_interact.pep.xls
20170410_JA4_01_quant_interact.pep.xlsx
20170531_ETNP_JA14_01_PTMopt_quant_interact.pep.xls
20170531_ETNP_JA14_01_PTMopt_quant_interact.pep.xlsx
20181003_4-19_965m_top_PTMopt_quant_interact.pep.xls
20181004_4-19_265m_topP_278_90min_PTMopt_quant_interact.pep.xls
20181004_4-19_265m_topP_278_90min_PTMopt_quant_interact.pep.xlsx
20181214_378_etnp2017_100m_trap_PTMopt_quant_interact.pep.xls
20181214_378_etnp2017_100m_trap_PTMopt_quant_interact.pep.xlsx
ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_Comet3_AA_NAAF.csv
ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_Comet3_totals.csv
ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_Comet50_totals.csv
ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_Comet50_ulfiltered.csv
ETNP-SKQ17-233-265m-0.3-JA4_PTMopt_Comet3_AA_NAAF.csv
ETNP-SKQ17-233-265m-0.3-JA4_PTMopt_Comet3_totals.csv
ETNP-SKQ17-233-265m-0.3-JA4_PTMopt_Comet50_ulfiltered.csv
ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet3_AA_NAAF.csv
ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet3_totals.csv
ETNP-SKQ17-243-9

## 231: 100 m McLane pump filtered on 0.3 um GF-75

In [4]:
# read the CSV into a dataframe using the pandas read_csv function
#cometdup278 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTMopt-hyroxylation/JA2_PTMopt_interact_quant_nopro.pep.csv", index_col='spectrum')

cometdup278 = pd.read_excel("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTMopt-hyroxylation/20181004_4-19_265m_topP_278_90min_PTMopt_quant_interact.pep.xlsx")

# remove redundant rows
comet278 = pd.DataFrame.drop_duplicates(cometdup278)


print("# redundant Comet peptides in combined dataframe", len(cometdup278))
print("# nonredundant Comet peptides in combined dataframe", len(comet278))

#look at the dataframe
comet278.head()

# redundant Comet peptides in combined dataframe 32874
# nonredundant Comet peptides in combined dataframe 32874


,spectrum,xcorr,deltacn,expect,ions,peptide,protein,calc_neutral_pep_mass,precursor_intensity,protein_descr
0,20181004_4-19_265m_topP_278_90min.45087.45087.2,5.642,0.576,1.260000e-09,27/44,R.AIQQQIENPLAQQILSGELVPGK.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,2473.3540,2215090.0,RecName: Full=Chaperone protein ClpB; AltName:...
1,20181004_4-19_265m_topP_278_90min.45016.45016.3,5.634,0.462,1.970000e-05,29/88,R.AIQQQIENPLAQQILSGELVPGK.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,2473.3540,13100100.0,RecName: Full=Chaperone protein ClpB; AltName:...
2,20181004_4-19_265m_topP_278_90min.24694.24694.2,4.085,0.420,3.130000e-05,22/36,R.LPQVEGTGGDVQPSQDLVR.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,1994.0069,18697900.0,RecName: Full=Chaperone protein ClpB; AltName:...
3,20181004_4-19_265m_topP_278_90min.25494.25494.2,3.972,0.463,8.960000e-07,22/36,R.LPQVEGTGGDVQ[129.04]PSQDLVR.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,1994.9909,2532930.0,RecName: Full=Chaperone protein ClpB; AltName:...
4,20181004_4-19_265m_topP_278_90min.45263.45263.3,3.884,1.000,5.930000e-12,19/88,R.AIQQQIENPLAQQILSGELVPGK.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,2473.3540,376172.0,RecName: Full=Chaperone protein ClpB; AltName:...


In [5]:
# get rid of rows where the xcorr is unavailable (usually 3 or so)
comet278 = comet278[comet278.xcorr != '[unavailable]']

# use str.strip with indexing by str[0] to add a column with the peptide's left terminus
comet278['L terminus'] = comet278['peptide'].astype(str).str[0]

# use str.strip with indexing by str[-1] to add a column with the peptide's left terminus
comet278['R terminus'] = comet278['peptide'].str.strip().str[-1]

# use a count function to enumerate the # of A's (alanines) in each peptide
comet278['A'] = comet278['peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
comet278['C'] = comet278['peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
comet278['D'] = comet278['peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
comet278['E'] = comet278['peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
comet278['F'] = comet278['peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
comet278['G'] = comet278['peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
comet278['H'] = comet278['peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in comet278 output, there will be no isoleucines (they're lumped in with leucines)
comet278['I'] = comet278['peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
comet278['K'] = comet278['peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
comet278['L'] = comet278['peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
comet278['M'] = comet278['peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
comet278['N'] = comet278['peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
comet278['P'] = comet278['peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
comet278['Q'] = comet278['peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
comet278['R'] = comet278['peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
comet278['S'] = comet278['peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
comet278['T'] = comet278['peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
comet278['V'] = comet278['peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
comet278['W'] = comet278['peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
comet278['Y'] = comet278['peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
comet278['c-carb'] = comet278['peptide'].str.count("160.03")

# use a count function to enumerate the # of oxidized M's in each peptide
comet278['m-oxid'] = comet278['peptide'].str.count("147.04")

# use a count function to enumerate the # of deamidated N's in each peptide
comet278['n-deam'] = comet278['peptide'].str.count("115.03")

# use a count function to enumerate the # of deamidated Q's in each peptide
comet278['q-deam'] = comet278['peptide'].str.count("129.04")

# use a count function to enumerate the # of hydroxylated K's in each peptide
comet278['k-hydr'] = comet278['peptide'].str.count("144.09")

# use a count function to enumerate the # of hydroxylated P's in each peptide
comet278['p-hydr'] = comet278['peptide'].str.count("131.05")

# use a count function to enumerate the # of methylated R's in each peptide
comet278['r-meth'] = comet278['peptide'].str.count("170.12")

# create a column with 'stripped' peptide sequences using strip
comet278['stripped peptide'] = comet278['peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
comet278['stripped length'] = comet278['stripped peptide'].apply(len)

comet278['NAAF num.'] = comet278['precursor_intensity'] / comet278['stripped length']

# total the number of modifications in sequence
comet278['ptm-total'] = comet278['c-carb'] + comet278['m-oxid'] + comet278['n-deam'] + comet278['q-deam'] + comet278['k-hydr'] + comet278['p-hydr'] + comet278['r-meth']

# write modified dataframe to new txt file
comet278.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTMopt-hyroxylation/ETNP-SKQ17-278-265m-trap_PTMopt_Comet50_ulfiltered.csv")

# check out the results
comet278.head()

/home/millieginty/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,spectrum,xcorr,deltacn,expect,ions,peptide,protein,calc_neutral_pep_mass,precursor_intensity,protein_descr,...,m-oxid,n-deam,q-deam,k-hydr,p-hydr,r-meth,stripped peptide,stripped length,NAAF num.,ptm-total
0,20181004_4-19_265m_topP_278_90min.45087.45087.2,5.642,0.576,1.260000e-09,27/44,R.AIQQQIENPLAQQILSGELVPGK.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,2473.3540,2215090.0,RecName: Full=Chaperone protein ClpB; AltName:...,...,0,0,0,0,0,0,R.AIQQQIENPLAQQILSGELVPGK.V,27,82040.370370,0
1,20181004_4-19_265m_topP_278_90min.45016.45016.3,5.634,0.462,1.970000e-05,29/88,R.AIQQQIENPLAQQILSGELVPGK.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,2473.3540,13100100.0,RecName: Full=Chaperone protein ClpB; AltName:...,...,0,0,0,0,0,0,R.AIQQQIENPLAQQILSGELVPGK.V,27,485188.888889,0
2,20181004_4-19_265m_topP_278_90min.24694.24694.2,4.085,0.420,3.130000e-05,22/36,R.LPQVEGTGGDVQPSQDLVR.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,1994.0069,18697900.0,RecName: Full=Chaperone protein ClpB; AltName:...,...,0,0,0,0,0,0,R.LPQVEGTGGDVQPSQDLVR.V,23,812952.173913,0
3,20181004_4-19_265m_topP_278_90min.25494.25494.2,3.972,0.463,8.960000e-07,22/36,R.LPQVEGTGGDVQ[129.04]PSQDLVR.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,1994.9909,2532930.0,RecName: Full=Chaperone protein ClpB; AltName:...,...,0,0,1,0,0,0,R.LPQVEGTGGDVQ[129.04]PSQDLVR.V,31,81707.419355,1
4,20181004_4-19_265m_topP_278_90min.45263.45263.3,3.884,1.000,5.930000e-12,19/88,R.AIQQQIENPLAQQILSGELVPGK.V,gi|54036848|sp|P63284.1|CLPB_ECOLI,2473.3540,376172.0,RecName: Full=Chaperone protein ClpB; AltName:...,...,0,0,0,0,0,0,R.AIQQQIENPLAQQILSGELVPGK.V,27,13932.296296,0


## Calculating the false discovery rate (% FDR)

### Filtering PSMs > a selected XCorr value and exporting peptides

In [11]:
# Let's separate out the decoy hits from the good ones

comet278pmm = comet278[~comet278['protein'].str.contains("DECOY")]
comet278dec = comet278[comet278['protein'].str.contains("DECOY")]

# how many PSM that are only PMM (proteins in the database)?

print("# real Comet PSMs", len(comet278pmm))

# compared to how many PSMs containing decoys?

print("# decoy Comet PSMs", len(comet278dec))

# calculate the bulk FDR (all PSMs so let's not beat ourselves up)

r = len(comet278pmm)
d = len(comet278dec)

FDR = d/r*100

print("False discovery rate = ", FDR)

TypeError: bad operand type for unary ~: 'float'

In [8]:
# keep only peptides  >3 XCorr
# need to convert Xcorr column from strings to numeric so we can use loc
comet278['xcorr'] = pd.to_numeric(comet278['xcorr'])

comet278_3 = comet278.loc[comet278['xcorr'] >= 3]

# What's the FDR?

# Let's separate out the decoy hits from the good ones

comet278pmm3 = comet278_3[~comet278_3['protein'].str.contains("DECOY")]
comet278dec3 = comet278_3[comet278_3['protein'].str.contains("DECOY")]

# how many PSM that are only PMM (proteins in the database)?

print("# real Comet PSMs", len(comet278pmm3))

# compared to how many PSMs containing decoys?

print("# decoy Comet PSMs", len(comet278dec3))

# calculate the FDR 

r = len(comet278pmm3)
d = len(comet278dec3)

FDR = d/(d+r)*100

print("False discovery rate = ", FDR)

# real Comet PSMs 19
# decoy Comet PSMs 0
False discovery rate =  0.0


## NAAF correction and exporting files with AA and PTM totals:

### Only for DECOY-filtered Comet PSMs > XCorr 3.0

In [9]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >XCorr 3)

index = ['sample total']

data = {'A': comet278pmm3['A'].sum(),
        'C': comet278pmm3['C'].sum(),
        'D': comet278pmm3['D'].sum(),
        'E': comet278pmm3['E'].sum(),
        'F': comet278pmm3['F'].sum(),
        'G': comet278pmm3['G'].sum(),
        'H': comet278pmm3['H'].sum(),
        'I': comet278pmm3['I'].sum(),
        'K': comet278pmm3['K'].sum(),
        'I': comet278pmm3['I'].sum(),
        'L': comet278pmm3['L'].sum(),
        'M': comet278pmm3['M'].sum(),
        'N': comet278pmm3['N'].sum(),
        'P': comet278pmm3['P'].sum(),
        'Q': comet278pmm3['Q'].sum(),
        'R': comet278pmm3['R'].sum(),
        'S': comet278pmm3['S'].sum(),
        'T': comet278pmm3['T'].sum(),
        'V': comet278pmm3['V'].sum(),
        'W': comet278pmm3['W'].sum(),
        'Y': comet278pmm3['Y'].sum(),
        'c-carb': comet278pmm3['c-carb'].sum(),
        'm-oxid': comet278pmm3['m-oxid'].sum(),
        'n-deam': comet278pmm3['n-deam'].sum(),
        'q-deam': comet278pmm3['q-deam'].sum(),
        'k-hydr': comet278pmm3['k-hydr'].sum(),
        'p-hydr': comet278pmm3['p-hydr'].sum(),
        'r-meth': comet278pmm3['r-meth'].sum(),
        'Total area': comet278pmm3['precursor_intensity'].sum(),
        'Total length': comet278pmm3['stripped length'].sum()
       }

totalcomet278pmm3 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'I', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalcomet278pmm3['% C w/ carb'] = totalcomet278pmm3['c-carb'] / totalcomet278pmm3['C'] 

# calculate percentage of M's that are oxidized
totalcomet278pmm3['% M w/ oxid'] = totalcomet278pmm3['m-oxid'] / totalcomet278pmm3['M'] 

# calculate percentage of N's that are deamidated
totalcomet278pmm3['% N w/ deam'] = totalcomet278pmm3['n-deam'] / totalcomet278pmm3['N'] 

# calculate percentage of Q's that are deamidated
totalcomet278pmm3['% Q w/ deam'] = totalcomet278pmm3['q-deam'] / totalcomet278pmm3['Q'] 

# calculate percentage of K's that are hydroxylated
totalcomet278pmm3['% K w/ hydr'] = totalcomet278pmm3['k-hydr'] / totalcomet278pmm3['K'] 

# calculate percentage of P's that are hydroxylated
totalcomet278pmm3['% P w/ hydr'] = totalcomet278pmm3['p-hydr'] / totalcomet278pmm3['P'] 

# calculate percentage of R's that are methylated
totalcomet278pmm3['% R w/ meth'] = totalcomet278pmm3['r-meth'] / totalcomet278pmm3['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalcomet278pmm3['NAAF denom.'] = totalcomet278pmm3['Total area'] / totalcomet278pmm3['Total length']

# write modified dataframe to new txt file
totalcomet278pmm3.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTMopt-hyroxylation/ETNP-SKQ17-278-265m-trap_PTMopt_Comet3_totals.csv")

totalcomet278pmm3.head()

,A,C,D,E,F,G,H,I,K,I,...,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% P w/ hydr,% R w/ meth,NAAF denom.
sample total,29,0,17,27,3,34,2,25,11,25,...,71579687.0,437,NaN,0.0,0.0,0.052632,0.0,0.0,0.035714,163797.910755


In [10]:
# use the calculated NAAF factor (in totalcomet278pmm3 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF3 = 163797.910755

# use NAAF >XCorr 3 to get NAAF
comet278pmm3['NAAF factor'] = (comet278pmm3['NAAF num.'])/NAAF3

# make a dataframe that contains only what we need: sequences, AAs, PTMs
comet278pmm3_AA = comet278pmm3[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', \
                                'q-deam', 'k-hydr', 'p-hydr', 'r-meth']].copy()

# multiply the NAAF3 factor by the AAs to normalize its abundance by peak area and peptide length

comet278pmm3_AA['A-NAAF3'] = comet278pmm3_AA['A'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['C-NAAF3'] = comet278pmm3_AA['C'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['D-NAAF3'] = comet278pmm3_AA['D'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['E-NAAF3'] = comet278pmm3_AA['E'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['F-NAAF3'] = comet278pmm3_AA['F'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['G-NAAF3'] = comet278pmm3_AA['G'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['H-NAAF3'] = comet278pmm3_AA['H'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['K-NAAF3'] = comet278pmm3_AA['K'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['I-NAAF3'] = comet278pmm3_AA['I'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['L-NAAF3'] = comet278pmm3_AA['L'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['M-NAAF3'] = comet278pmm3_AA['M'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['N-NAAF3'] = comet278pmm3_AA['N'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['P-NAAF3'] = comet278pmm3_AA['P'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['Q-NAAF3'] = comet278pmm3_AA['Q'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['R-NAAF3'] = comet278pmm3_AA['R'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['S-NAAF3'] = comet278pmm3_AA['S'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['T-NAAF3'] = comet278pmm3_AA['T'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['V-NAAF3'] = comet278pmm3_AA['V'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['W-NAAF3'] = comet278pmm3_AA['W'] * comet278pmm3['NAAF factor']
comet278pmm3_AA['Y-NAAF3'] = comet278pmm3_AA['Y'] * comet278pmm3['NAAF factor']

# multiply the NAAF3 factor by the PTMs normalize its abundance by peak area and peptide length

comet278pmm3_AA['ccarb-NAAF3'] = comet278pmm3_AA['c-carb'] * comet278pmm3_AA['NAAF factor']
comet278pmm3_AA['moxid-NAAF3'] = comet278pmm3_AA['m-oxid'] * comet278pmm3_AA['NAAF factor']
comet278pmm3_AA['ndeam-NAAF3'] = comet278pmm3_AA['n-deam'] * comet278pmm3_AA['NAAF factor']
comet278pmm3_AA['qdeam-NAAF3'] = comet278pmm3_AA['q-deam'] * comet278pmm3_AA['NAAF factor']
comet278pmm3_AA['khydr-NAAF3'] = comet278pmm3_AA['k-hydr'] * comet278pmm3_AA['NAAF factor']
comet278pmm3_AA['phydr-NAAF3'] = comet278pmm3_AA['p-hydr'] * comet278pmm3_AA['NAAF factor']
comet278pmm3_AA['rmeth-NAAF3'] = comet278pmm3_AA['r-meth'] * comet278pmm3_AA['NAAF factor']

# write the dataframe to a new csv
comet278pmm3_AA.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTMopt-hyroxylation//ETNP-SKQ17-278-265m-trap_PTMopt_Comet3_AA_NAAF.csv")

comet278pmm3_AA.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,K,...,V-NAAF3,W-NAAF3,Y-NAAF3,ccarb-NAAF3,moxid-NAAF3,ndeam-NAAF3,qdeam-NAAF3,khydr-NAAF3,phydr-NAAF3,rmeth-NAAF3
0,R.AIQQQIENPLAQQILSGELVPGK.V,0.500863,2,0,0,2,0,2,0,1,...,1.001727,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,R.AIQQQIENPLAQQILSGELVPGK.V,2.962119,2,0,0,2,0,2,0,1,...,5.924238,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,R.LPQVEGTGGDVQPSQDLVR.V,4.963141,0,0,2,1,0,3,0,0,...,19.852565,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,R.LPQVEGTGGDVQ[129.04]PSQDLVR.V,0.498831,0,0,2,1,0,3,0,0,...,1.995323,0.0,0.0,0.0,0.0,0.0,0.498831,0.0,0.0,0.0
4,R.AIQQQIENPLAQQILSGELVPGK.V,0.085058,2,0,0,2,0,2,0,1,...,0.170116,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


### Visualizing the results

In [ ]:
# take only AA totals and transpose for easier bar plotting in matplotlib

comet231aatot = totalcomet231pmm3[['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']].copy().T

# take only PTMs and transpose for easier bar plotting in matplotlib

#comet231ptmtot = totalcomet231pmm3[['% C w/ carb.', '% M w/ oxid', '% N w/ deam', '% Q w/ deam', '% K w/ hydr', '% P w/ hydr', '% R w/ meth']].copy().T

In [ ]:
# bar plot of residue totals

x_labels = ['sample total']

ax = comet231aatot.plot(y=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], kind="bar", title = '100 m suspended')
plt.xticks(rotation=0)
ax.get_legend().remove()
ax.set_xticklabels(x_labels)

In [ ]:
# bar plot of residue totals
# there is no isoleucine (I) in Peaks data, which is why L is really big and I is 0

my_colors = [(x/10.0, x/20.0, 0.75) for x in range(len(peaksaatot))] # <-- Quick gradient example along the Red/Green dimensions.

ax = peaksaatot.plot(y=['sample total'], kind="bar", color = 'green', title = '100 m suspended')


In [ ]:
# bar plot of relative modifications

ax = totalpeaks.plot(y=['% C w/ carb.', '% M w/ oxid', '% N w/ deam', '% Q w/ deam', '% K w/ hydr', '% P w/ hydr', '% K w/ meth', '% R w/ meth'], kind="bar", title = '100 m suspended')
ax.set_xticklabels([])

In [ ]:
# bar plot of relative mods


ax = peaksreltot.plot(y=['sample total'], kind="bar", title = '100 m suspended')
plt.xticks(rotation=45)

In [ ]:
# making evenly spaced bins for the ALC data based on the min and max, called above
bins = [50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
labels = ['50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100']

# use pandas cut function to do the binning itself
peaks['binned'] = pd.cut(peaks['ALC (%)'], bins=bins, labels=labels)

# bar plots of binned PTM data

index = ['50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100']
data = {'Total PTMs': [peaks.groupby('binned')['ptm-total'].sum()['50-55'], peaks.groupby('binned')['ptm-total'].sum()['55-60'], peaks.groupby('binned')['ptm-total'].sum()['60-65'], peaks.groupby('binned')['ptm-total'].sum()['65-70'], peaks.groupby('binned')['ptm-total'].sum()['70-75'], peaks.groupby('binned')['ptm-total'].sum()['75-80'], peaks.groupby('binned')['ptm-total'].sum()['80-85'], peaks.groupby('binned')['ptm-total'].sum()['85-90'], peaks.groupby('binned')['ptm-total'].sum()['90-95'], peaks.groupby('binned')['ptm-total'].sum()['95-100']],
        'Cys carb.': [peaks.groupby('binned')['c-carb'].sum()['50-55'], peaks.groupby('binned')['c-carb'].sum()['55-60'], peaks.groupby('binned')['c-carb'].sum()['60-65'], peaks.groupby('binned')['c-carb'].sum()['65-70'], peaks.groupby('binned')['c-carb'].sum()['70-75'], peaks.groupby('binned')['c-carb'].sum()['75-80'], peaks.groupby('binned')['c-carb'].sum()['80-85'], peaks.groupby('binned')['c-carb'].sum()['85-90'], peaks.groupby('binned')['c-carb'].sum()['90-95'], peaks.groupby('binned')['c-carb'].sum()['95-100']],
        'Met oxi.': [peaks.groupby('binned')['m-oxid'].sum()['50-55'], peaks.groupby('binned')['m-oxid'].sum()['55-60'], peaks.groupby('binned')['m-oxid'].sum()['60-65'], peaks.groupby('binned')['m-oxid'].sum()['65-70'], peaks.groupby('binned')['m-oxid'].sum()['70-75'], peaks.groupby('binned')['m-oxid'].sum()['75-80'], peaks.groupby('binned')['m-oxid'].sum()['80-85'], peaks.groupby('binned')['m-oxid'].sum()['85-90'], peaks.groupby('binned')['m-oxid'].sum()['90-95'], peaks.groupby('binned')['m-oxid'].sum()['95-100']],
        'Asp deam.': [peaks.groupby('binned')['n-deam'].sum()['50-55'], peaks.groupby('binned')['n-deam'].sum()['55-60'], peaks.groupby('binned')['n-deam'].sum()['60-65'], peaks.groupby('binned')['n-deam'].sum()['65-70'], peaks.groupby('binned')['n-deam'].sum()['70-75'], peaks.groupby('binned')['n-deam'].sum()['75-80'], peaks.groupby('binned')['n-deam'].sum()['80-85'], peaks.groupby('binned')['n-deam'].sum()['85-90'], peaks.groupby('binned')['n-deam'].sum()['90-95'], peaks.groupby('binned')['n-deam'].sum()['95-100']],
        'Glut deam.': [peaks.groupby('binned')['q-deam'].sum()['50-55'], peaks.groupby('binned')['q-deam'].sum()['55-60'], peaks.groupby('binned')['q-deam'].sum()['60-65'], peaks.groupby('binned')['q-deam'].sum()['65-70'], peaks.groupby('binned')['q-deam'].sum()['70-75'], peaks.groupby('binned')['q-deam'].sum()['75-80'], peaks.groupby('binned')['q-deam'].sum()['80-85'], peaks.groupby('binned')['q-deam'].sum()['85-90'], peaks.groupby('binned')['q-deam'].sum()['90-95'], peaks.groupby('binned')['q-deam'].sum()['95-100']],
        'Lys hydr': [peaks.groupby('binned')['k-hydr'].sum()['50-55'], peaks.groupby('binned')['k-hydr'].sum()['55-60'], peaks.groupby('binned')['k-hydr'].sum()['60-65'], peaks.groupby('binned')['k-hydr'].sum()['65-70'], peaks.groupby('binned')['k-hydr'].sum()['70-75'], peaks.groupby('binned')['k-hydr'].sum()['75-80'], peaks.groupby('binned')['k-hydr'].sum()['80-85'], peaks.groupby('binned')['k-hydr'].sum()['85-90'], peaks.groupby('binned')['k-hydr'].sum()['90-95'], peaks.groupby('binned')['k-hydr'].sum()['95-100']],
        'Pro hydr': [peaks.groupby('binned')['p-hydr'].sum()['50-55'], peaks.groupby('binned')['p-hydr'].sum()['55-60'], peaks.groupby('binned')['p-hydr'].sum()['60-65'], peaks.groupby('binned')['p-hydr'].sum()['65-70'], peaks.groupby('binned')['p-hydr'].sum()['70-75'], peaks.groupby('binned')['p-hydr'].sum()['75-80'], peaks.groupby('binned')['p-hydr'].sum()['80-85'], peaks.groupby('binned')['p-hydr'].sum()['85-90'], peaks.groupby('binned')['p-hydr'].sum()['90-95'], peaks.groupby('binned')['p-hydr'].sum()['95-100']],
        'Lys meth.': [peaks.groupby('binned')['k-meth'].sum()['50-55'], peaks.groupby('binned')['k-meth'].sum()['55-60'], peaks.groupby('binned')['k-meth'].sum()['60-65'], peaks.groupby('binned')['k-meth'].sum()['65-70'], peaks.groupby('binned')['k-meth'].sum()['70-75'], peaks.groupby('binned')['k-meth'].sum()['75-80'], peaks.groupby('binned')['k-meth'].sum()['80-85'], peaks.groupby('binned')['k-meth'].sum()['85-90'], peaks.groupby('binned')['k-meth'].sum()['90-95'], peaks.groupby('binned')['k-meth'].sum()['95-100']],
        'Arg meth.': [peaks.groupby('binned')['r-meth'].sum()['50-55'], peaks.groupby('binned')['r-meth'].sum()['55-60'], peaks.groupby('binned')['r-meth'].sum()['60-65'], peaks.groupby('binned')['r-meth'].sum()['65-70'], peaks.groupby('binned')['r-meth'].sum()['70-75'], peaks.groupby('binned')['r-meth'].sum()['75-80'], peaks.groupby('binned')['r-meth'].sum()['80-85'], peaks.groupby('binned')['r-meth'].sum()['85-90'], peaks.groupby('binned')['r-meth'].sum()['90-95'], peaks.groupby('binned')['r-meth'].sum()['95-100']]
        }

peaksbin = pd.DataFrame(data, columns=['Total PTMs','Cys carb.','Met oxi.','Asp deam.', 'Glut deam.', 'Lys hydr', 'Pro hydr', 'Lys meth.', 'Arg meth.'], index=index)

# write the peaks bin ptm dataframe to a csv:
peaksbin.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_ptm.csv")

ax1 = peaksbin.plot.bar(y='Total PTMs', rot=45)
ax1.set_title('Total PTMs')

ax2 = peaksbin.plot.bar(y='Cys carb.', rot=45)
ax2.set_title('Cysteine carbamidomethylation')

ax3 = peaksbin.plot.bar(y='Met oxi.', rot=45)
ax3.set_title('Methionine oxidation')

ax4 = peaksbin.plot.bar(y='Asp deam.', rot=45)
ax4.set_title('Asparagine deamidation')

ax5 = peaksbin.plot.bar(y='Glut deam.', rot=45)
ax5.set_title('Glutamine deamidation')

ax6 = peaksbin.plot.bar(y='Lys hydr', rot=45)
ax6.set_title('Lysine hydroxylation')

ax7 = peaksbin.plot.bar(y='Pro hydr', rot=45)
ax7.set_title('Proline hydroxylation')

ax8 = peaksbin.plot.bar(y='Lys meth.', rot=45)
ax8.set_title('Lysine methylation')

ax9 = peaksbin.plot.bar(y='Arg meth.', rot=45)
ax9.set_title('Arginine methylation')


### Exporting txt files of stripped peptides at confidence cutoffs:

In [ ]:
# keep only peptide column >50% ALC
pep = peaks[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.txt > /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas

# look

print("# of DN peptide >50% ALC", len(pep))
pep.head()

In [ ]:
# keep only peptides  >80% ALC
peaks80 = peaks.loc[peaks['ALC (%)'] >= 80]

# see how many rows and double check
# peaks80.head(-10)

# keep only peptide column 
pep80 = peaks80[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN80_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN80_stripped_peptides.txt > /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN80_stripped_peptides.fas

print("# of DN peptide >80% ALC", len(pep80))
pep80.head()

### Using BioPython to query peptide sequences

I installed the BioPython package using `pip install biopython`. All instructions and information [here](https://www.tutorialspoint.com/biopython/index.htm). 

GitHub project: https://github.com/biopython/biopython

I'm relying on the ProtParam module to parse sequences for relative AA composition, instability, secondary structure, instability, and hydrophobicity. You can read more about that module and the studies the indecies are derived from here:

https://biopython.org/wiki/ProtParam

In [ ]:
# Bio.SeqIO is the standard Sequence Input/Output interface for BioPython 1.43 and later
# Bio.SeqIO provides a simple uniform interface to input and output assorted sequence file formats.
# (including multiple sequence alignments), but will only deal with sequences as SeqRecord objects

# for accepted file formats see https://biopython.org/wiki/SeqIO

from Bio import SeqIO
#for seq_record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/MED4/MED2_tryp_1raw_db_peptides_nmod.fasta", "fasta"):
    #print(seq_record.id)
    #print(repr(seq_record.seq))
    #print(len(seq_record))
    
# I commented the print functions out so the output doesn't take up too much space. 

In [ ]:
# seeing what the ProtParam module can do with a single protein sequence:

from Bio.SeqUtils.ProtParam import ProteinAnalysis

test_seq = "MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEECLFLERLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILFLPLPV"

analysed_seq = ProteinAnalysis(test_seq)
print("molecular weight of seq =", analysed_seq.molecular_weight())

# calculates the aromaticity value of a protein according to Lobry & Gautier (1994, Nucleic Acids Res., 22, 3174-3180). 
# it's simply the relative frequency of Phe+Trp+Tyr.

analysed_seq.aromaticity()
print("aromaticity of seq =", analysed_seq.aromaticity())

# secondary_structure_fraction:
# this methods returns a list of the fraction of amino acids which tend to be in helix, turn or sheet. 
# AAs in helix: V, I, Y, F, W, L
# AAs in turn: N, P, G, S
# AAs in sheet: E, M, A, L
# the returned list contains 3 values: [Helix, Turn, Sheet]

analysed_seq.secondary_structure_fraction()
print("frac in H T S =", analysed_seq.secondary_structure_fraction())

# the instability index, an implementation of the method of Guruprasad et al. (1990, Protein Engineering, 4, 155-161).
# this method tests a protein for stability. 
# any value above 40 means the protein is unstable (=has a short half life)
# NOT SURE WHAT THIS MEANS FOR PEPTIDES, BUT WE COULD DO THIS FOR PROTEINS

analysed_seq.instability_index()
print("instability =", analysed_seq.instability_index())

# count_amino_acids will do just that, and get_amino_acids_percent will return %'s for each AA across the sequence. 
analysed_seq.get_amino_acids_percent()

# taking the returned dictionary and converting to a dataframe

aadict = analysed_seq.get_amino_acids_percent()
aadf = pd.DataFrame(list(aadict.items()),columns = ['residue','% occurance']) 

aadf.head()

In [ ]:
# use SeqIO and a loop to apply count_amino_acids to each sequence in the file
# aatot will give us the total number of each residue in the entire sample output

import collections
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

all_aas = collections.defaultdict(int)
for record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas", "fasta"):
    x = ProteinAnalysis(str(record.seq))
    #print(record.id, x.count_amino_acids())
    for aa, count in x.count_amino_acids().items():
        all_aas[aa] += count        
        
# made a dataframe for amino acid total counts        
data = (all_aas)
aatot = pd.DataFrame(data, index = ['sample sequence total'])
aatot.head()

In [ ]:
from pandas import Series, DataFrame

with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas') as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2)).set_index(['ID'])

idseq.head()

In [ ]:
from pandas import Series, DataFrame



with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    aa = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        aa.count_amino_acids(seq_record.seq)
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')
s3 = Series(aa, name='AAs')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2, AAs=s3)).set_index(['ID'])

idseq.head()

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import ProtParam

handle = open("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas") 
for record in SeqIO.parse(handle, "fasta"): 
    seq = str(record.seq)
    X = ProtParam.ProteinAnalysis(seq)
    print(X.count_amino_acids()) 
    #print X.get_amino_acids_percent() 
    #print X.molecular_weight() 
    #print X.aromaticity() 
    #print X.instability_index() 
    #print X.flexibility() 
    #print X.isoelectric_point() 
    #print X.secondary_structure_fraction()
    
# made a data series from the count_amino_acids function
# aacount = {X.count_amino_acids()}

# made a pandas dataframe from the series generated above
# aacount = pd.DataFrame(list(data.items()),columns = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']) 

aacount = pd.DataFrame(X.count_amino_acids(), index=[0])

# look at new dataframe

# aacount.head()